In [1]:
import numpy as np
import pandas as pd
import operator
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from tqdm import tqdm
from collections import OrderedDict
import csv

In [3]:
import numpy as np
import pandas as pd
import operator
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from tqdm import tqdm
from collections import OrderedDict
import csv

base = 'ML-1M'
directory = "../data/ml-100k/u.data"
sep = '\t'
header = False
timestamp_exists = True

numRatings = sum(1 for line in open(directory))
usersId = np.zeros(numRatings, dtype=np.int32)
itemsId = np.zeros(numRatings, dtype=np.int16)
ratings = np.zeros(numRatings, dtype=np.int8)
timestamp = np.zeros(numRatings, dtype=np.int32)

# Leitura do arquivo
def get_max_uid_iid(directory):

	max_uid, max_iid = -1, -1
	file = open(directory, "r")
	if header: file.readline()
	for row in file:
		values = row.split(sep)
		uid, iid = int(values[0]),int(values[1])
		if uid > max_uid: max_uid = uid
		if iid > max_iid: max_iid = iid

	return max_uid, max_iid

def read_archive(directory):

	file = open(directory, "r")
	max_uid, max_iid = get_max_uid_iid(directory)
	
	backup_uid = np.full(max_uid+1, -1, dtype=np.int32)
	backup_iid = np.full(max_iid+1, -1, dtype=np.int32)
	
	cont, iid_seq, uid_seq = 0, 0, 0

	if header: file.readline()
	for row in file:
		
		values = row.split(sep)
		uid, iid, rating, ts = int(values[0]),int(values[1]),values[2], int(values[3].replace('\n', ''))
		
		if backup_uid[uid] == -1:
			backup_uid[uid] = uid_seq
			uid_seq += 1

		if backup_iid[iid] == -1:
			backup_iid[iid] = iid_seq
			iid_seq += 1

		usersId[cont] = backup_uid[uid]
		itemsId[cont] = backup_iid[iid]
		ratings[cont] = rating
		if timestamp_exists: timestamp[cont] = ts
		else: timestamp[cont] = cont
		cont += 1
		
	file.close()
	return usersId, itemsId, ratings, timestamp, backup_uid, backup_iid

In [5]:
usersId, itemsId, ratings, timestamp, backup_uid, backup_iid = read_archive(directory)

In [6]:
num_movies = len(np.unique(itemsId))
num_users = len(np.unique(usersId))
print(num_movies, num_users)

1682 943


In [15]:
ratings_matrix = csr_matrix((ratings, (usersId, itemsId)), shape=(num_users+1, num_movies+1))
timestamp_matrix = csr_matrix((timestamp, (usersId, itemsId)), shape=(num_users+1, num_movies+1))

In [9]:
backup_uid[196]

0

In [17]:
backup_iid[251]

522

In [18]:
timestamp_matrix[0,522]

881251274

 ### Base: ML-1M

In [ ]:
np.N

In [2]:
# numRatings = 1000209
numRatings = 5976479

In [3]:
usersId = np.zeros(numRatings, dtype=np.int32)
itemsId = np.zeros(numRatings, dtype=np.int16)
ratings = np.zeros(numRatings, dtype=np.int8)
timestamp = np.zeros(numRatings, dtype=np.int32)

### Leitura do arquivo

In [5]:
# file = open("../data/ml-1m/ratings.dat", "r")
file = open("../data/GoodBooks/ratings.csv", "r")
# file.readline()
backupId = {}
cont = 0
iid = 1
for row in file:
    
    values = row.split(',')
    usersId[cont] = int(values[0])
    
    if int(values[1]) not in backupId:
        backupId[int(values[1])] = iid
        iid+=1
    
    itemsId[cont] = backupId[int(values[1])]
    ratings[cont] = int(values[2])
#     timestamp[cont] = int(values[3].replace('\n', ''))
    timestamp[cont] = cont
    cont += 1
    
file.close()

In [6]:
numMovies = len(np.unique(itemsId))
numUsers = len(np.unique(usersId))
print("Número de usuários: ", numUsers)
print("Número de filmes: ", numMovies)

Número de usuários:  53424
Número de filmes:  10000


### Criando as matrizes de ratings e timestamp

In [7]:
# matrix of ratings
ratings_matrix = csr_matrix((ratings, (usersId, itemsId)), shape=(numUsers+1, numMovies+1))

# matrix of timestamp
timestamp_matrix = csr_matrix((timestamp, (usersId, itemsId)), shape=(numUsers+1, numMovies+1))

users_dataset = np.unique(usersId)
print("Número de usuários: ", len(users_dataset))
print("Matriz de ratings: ", ratings_matrix.shape)
print("Matriz de timestamp: ", timestamp_matrix.shape)

Número de usuários:  53424
Matriz de ratings:  (53425, 10001)
Matriz de timestamp:  (53425, 10001)


### Ordenando os usuários pelo timestamp

In [13]:
user_access = {}
for u in users_dataset:
    user_access[u] = np.min(timestamp_matrix[u,:].data)
    
dict_sorted = OrderedDict(sorted(user_access.items(), key=operator.itemgetter(1), reverse = True))
users_order = list(dict_sorted.keys())

users_train = []
users_test = []

num_user_test = 0
for i in users_order:
#     print(len(ratings_matrix[i].data))
    if len(ratings_matrix[i].data) >= 120 and num_user_test < 10685: 
        users_test.append(i)
        num_user_test += 1
    else: users_train.append(i)
        
print("Número de usuários para o treino: ", len(users_train))
print("Número dos novos usuários selecionados: :", len(users_test))

Número de usuários para o treino:  42739
Número dos novos usuários selecionados: : 10685


In [14]:
num_user_test

10685

### Separando o treino e teste

In [15]:
# Selecting the last users to join to the system
# users_train = users_order[0:nb_users]
# users_test = users_order[nb_users:len(users_dataset)]

print("Users in train and test sets:")
print("Train:", len(users_train))
print("Test:", len(users_test))
print("Total:", len(users_train) + len(users_test))

Users in train and test sets:
Train: 42739
Test: 10685
Total: 53424


In [17]:
numRatings

5976479

In [18]:
pos_train = []
pos_test = []

for itr in tqdm(range(0,numRatings), position=0, leave=True):
    # selecting for train or test
    if (usersId[itr] in users_train):
        pos_train.append(itr)
    elif (usersId[itr] in users_test):
        pos_test.append(itr)
    else:
        print("Error: User ["+str(usersId[itr])+"] does not identified.")


print(len(pos_train), " <> ", len(pos_test))

# Spliting the sets
matrix_train = np.zeros((len(pos_train),4), dtype = int)
matrix_test = np.zeros((len(pos_test),4))

# train set
matrix_train[:,0] = usersId[pos_train[:]]
matrix_train[:,1] = itemsId[pos_train[:]]
matrix_train[:,2] = ratings[pos_train[:]]
matrix_train[:,3] = timestamp[pos_train[:]]

# test set
matrix_test[:,0] = usersId[pos_test[:]]
matrix_test[:,1] = itemsId[pos_test[:]]
matrix_test[:,2] = ratings[pos_test[:]]
matrix_test[:,3] = timestamp[pos_test[:]]

matrix_train.shape
matrix_test.shape

100%|██████████| 5976479/5976479 [1:19:44<00:00, 1249.09it/s]


4501704  <>  1474775


(1474775, 4)

In [19]:
print(matrix_train.shape, matrix_test.shape)

(488025, 4) (512184, 4)


### Salvando o treino e teste

In [19]:
# Saving into files
np.savetxt("trainSet_good-books.data", 
		   matrix_train, delimiter="::", header = str("userId::itemId::rating::timestamp"), fmt = '%d')
np.savetxt("testSet_goog-books.data", 
		   matrix_test, delimiter="::", header = str("userId::itemId::rating::timestamp"), fmt = '%d')
    